<a href="https://colab.research.google.com/github/ZidanMau24/AMS2022/blob/main/cleaning_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import string
import time
from copy import deepcopy
import pandas as pd

In [ ]:
df = pd.read_csv('PrayForKanjuruhan-tweets.csv')

In [ ]:
df.head()

,Datetime,Tweet Id,Text,Source,Likes,RT
0,2022-10-04 03:23:28,1577137299858391040,RT @dialoglukaku: Buat temen-temen boleh bantu...,Twitter for Android,0,2302
1,2022-10-04 03:23:26,1577137290735861763,"RT @NarasiNewsroom: Mon maap, syp yg bahagia y...",Twitter for Android,0,5696
2,2022-10-04 03:23:26,1577137288395358209,RT @GOAL_ID: 𝗥𝗲𝘀𝗽𝗲𝗰𝘁 @realmadriden @osasuna_en...,Twitter for Android,0,47
3,2022-10-04 03:23:25,1577137287741046784,RT @sulaimaneffendi: Dari Surabaya untuk Malan...,Twitter for iPhone,0,244
4,2022-10-04 03:23:25,1577137286348611587,RT @GOAL_ID: 𝗥𝗲𝘀𝗽𝗲𝗰𝘁 @realmadriden @osasuna_en...,Twitter for Android,0,47


In [ ]:
!pip install ekphrasis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.4 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 


In [ ]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",'emphasis', 'censored'},
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [ ]:
# panggil ekphrasis

def bersih_data(text):
    return " ".join(text_processor.pre_process_doc(text))

In [ ]:
# fungsi dari AMS 01-03. silakan cek bagaimana saya merubah menjadi fungsi

def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

def remove_space_alzami(text):
    return " ".join(text.split())

def remove_emoji_alzami(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

def remove_tab2(text):
    return re.sub('\s+',' ',text)

def remove_rt(text):
    return text.replace('RT'," ")

def remove_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())

def remove_incomplete_url(text):
    return text.replace("http://", " ").replace("https://", " ")

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def change_stripe(text):
    return text.replace('-'," ")

def lower(text):
    return text.lower()

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def lower(text):
    return text.lower()

def remove_whitespace_LT(text):
    return text.strip()

def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    return re.sub(pattern, "", text)

In [ ]:
# hapus untuk <>
def remove_number_eks(text):
    return text.replace('<number>'," ")

def remove_angka(text):
    return re.sub(r"\d+", "", text) 

def remove_URL_eks(text):
    return text.replace('URL'," ").replace('url'," ")

def space_punctuation(text):
    return re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', text)

In [ ]:
i = 0
final_string = []
s = ""
for text in df['Text'].values:
    filteredSentence = []
    EachReviewText = ""
    proc = remove_rt(text)
    proc = lower(proc)
    proc = change_stripe(proc)
    proc = remove_emoji_alzami(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = non_ascii(proc)
    proc = remove_incomplete_url(proc)
    proc = remove_excessive_dot(proc)
    proc = remove_whitespace_LT(proc)
    proc = remove_whitespace_multiple(proc)
    proc = remove_single_char(proc)
    proc = space_punctuation(proc)
    proc = remove_punctuation(proc)
    proc = remove_space_alzami(proc)
    proc = bersih_data(proc)
    proc = remove_number_eks(proc)
    proc = remove_angka(proc) 
    proc = remove_URL_eks(proc)
    EachReviewText = proc
    final_string.append(EachReviewText)

In [ ]:
df["cleanText"] = final_string

In [ ]:
df.head(
)

,Datetime,Tweet Id,Text,Source,Likes,RT,cleanText
0,2022-10-04 03:23:28,1577137299858391040,RT @dialoglukaku: Buat temen-temen boleh bantu...,Twitter for Android,0,2302,dialoglukaku buat temen temen boleh bantu retw...
1,2022-10-04 03:23:26,1577137290735861763,"RT @NarasiNewsroom: Mon maap, syp yg bahagia y...",Twitter for Android,0,5696,narasinewsroom mon maap syp yg bahagia pack kl...
2,2022-10-04 03:23:26,1577137288395358209,RT @GOAL_ID: 𝗥𝗲𝘀𝗽𝗲𝗰𝘁 @realmadriden @osasuna_en...,Twitter for Android,0,47,goal_id realmadriden osasuna_en coehahqlk
3,2022-10-04 03:23:25,1577137287741046784,RT @sulaimaneffendi: Dari Surabaya untuk Malan...,Twitter for iPhone,0,244,sulaimaneffendi dari surabaya untuk malang apa...
4,2022-10-04 03:23:25,1577137286348611587,RT @GOAL_ID: 𝗥𝗲𝘀𝗽𝗲𝗰𝘁 @realmadriden @osasuna_en...,Twitter for Android,0,47,goal_id realmadriden osasuna_en coehahqlk


In [ ]:
df.head()

,Datetime,Tweet Id,Text,Source,Likes,RT,cleanText
0,2022-10-04 03:23:28,1577137299858391040,RT @dialoglukaku: Buat temen-temen boleh bantu...,Twitter for Android,0,2302,dialoglukaku buat temen temen boleh bantu retw...
1,2022-10-04 03:23:26,1577137290735861763,"RT @NarasiNewsroom: Mon maap, syp yg bahagia y...",Twitter for Android,0,5696,narasinewsroom mon maap syp yg bahagia pack kl...
2,2022-10-04 03:23:26,1577137288395358209,RT @GOAL_ID: 𝗥𝗲𝘀𝗽𝗲𝗰𝘁 @realmadriden @osasuna_en...,Twitter for Android,0,47,goal_id realmadriden osasuna_en coehahqlk
3,2022-10-04 03:23:25,1577137287741046784,RT @sulaimaneffendi: Dari Surabaya untuk Malan...,Twitter for iPhone,0,244,sulaimaneffendi dari surabaya untuk malang apa...
4,2022-10-04 03:23:25,1577137286348611587,RT @GOAL_ID: 𝗥𝗲𝘀𝗽𝗲𝗰𝘁 @realmadriden @osasuna_en...,Twitter for Android,0,47,goal_id realmadriden osasuna_en coehahqlk


In [ ]:
#hapus data kosong
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Datetime   100 non-null    object
 1   Tweet Id   100 non-null    int64 
 2   Text       100 non-null    object
 3   Source     100 non-null    object
 4   Likes      100 non-null    int64 
 5   RT         100 non-null    int64 
 6   cleanText  100 non-null    object
dtypes: int64(3), object(4)
memory usage: 5.6+ KB


In [ ]:
df_hapus = df[~df['cleanText'].str.contains(" ")]

In [ ]:
df_hapus.head(10)

,Datetime,Tweet Id,Text,Source,Likes,RT,cleanText
58,2022-10-04 03:21:27,1577136789868802049,RT @Asroel_Boz: #PrayForKanjuruhan \n#StopPoli...,Twitter Web App,0,227,asroel_boz


In [ ]:
df_new = df[~df.isin(df_hapus)].dropna()

In [ ]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99 entries, 0 to 99
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Datetime   99 non-null     object 
 1   Tweet Id   99 non-null     float64
 2   Text       99 non-null     object 
 3   Source     99 non-null     object 
 4   Likes      99 non-null     float64
 5   RT         99 non-null     float64
 6   cleanText  99 non-null     object 
dtypes: float64(3), object(4)
memory usage: 6.2+ KB


In [ ]:
df_new

,Datetime,Tweet Id,Text,Source,Likes,RT,cleanText
0,2022-10-04 03:23:28,1.577137e+18,RT @dialoglukaku: Buat temen-temen boleh bantu...,Twitter for Android,0.0,2302.0,dialoglukaku buat temen temen boleh bantu retw...
1,2022-10-04 03:23:26,1.577137e+18,"RT @NarasiNewsroom: Mon maap, syp yg bahagia y...",Twitter for Android,0.0,5696.0,narasinewsroom mon maap syp yg bahagia pack kl...
2,2022-10-04 03:23:26,1.577137e+18,RT @GOAL_ID: 𝗥𝗲𝘀𝗽𝗲𝗰𝘁 @realmadriden @osasuna_en...,Twitter for Android,0.0,47.0,goal_id realmadriden osasuna_en coehahqlk
3,2022-10-04 03:23:25,1.577137e+18,RT @sulaimaneffendi: Dari Surabaya untuk Malan...,Twitter for iPhone,0.0,244.0,sulaimaneffendi dari surabaya untuk malang apa...
4,2022-10-04 03:23:25,1.577137e+18,RT @GOAL_ID: 𝗥𝗲𝘀𝗽𝗲𝗰𝘁 @realmadriden @osasuna_en...,Twitter for Android,0.0,47.0,goal_id realmadriden osasuna_en coehahqlk
...,...,...,...,...,...,...,...
95,2022-10-04 03:19:50,1.577136e+18,RT @christiyantosw: Coba perhatikan pukulan to...,Twitter for Android,0.0,219.0,christiyantosw coba perhatikan pukulan tongkat...
96,2022-10-04 03:19:47,1.577136e+18,RT @sulaimaneffendi: Dari Surabaya untuk Malan...,Twitter for Android,0.0,244.0,sulaimaneffendi dari surabaya untuk malang apa...
97,2022-10-04 03:19:46,1.577136e+18,RT @CoachJustinL: Ga ada gunanya cari kesalaha...,Twitter for Android,0.0,170.0,coachjustinl ga ada gunanya cari kesalahan ini...
98,2022-10-04 03:19:45,1.577136e+18,"RT @MprAldo: KETUM PSSI IWAN BULE : ""HADIRIN Y...",Twitter for Android,0.0,34.0,mpraldo ketum pssi iwan bule hadirin yang berb...


Normalisasi

In [ ]:
# token
import nltk
from nltk.tokenize import word_tokenize 

In [ ]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [ ]:
#download punkt karena error
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df_new['tokens'] = df['cleanText'].apply(word_tokenize_wrapper)

In [ ]:
df_new.head(10)

,Datetime,Tweet Id,Text,Source,Likes,RT,cleanText,tokens
0,2022-10-04 03:23:28,1.577137e+18,RT @dialoglukaku: Buat temen-temen boleh bantu...,Twitter for Android,0.0,2302.0,dialoglukaku buat temen temen boleh bantu retw...,"[dialoglukaku, buat, temen, temen, boleh, bant..."
1,2022-10-04 03:23:26,1.577137e+18,"RT @NarasiNewsroom: Mon maap, syp yg bahagia y...",Twitter for Android,0.0,5696.0,narasinewsroom mon maap syp yg bahagia pack kl...,"[narasinewsroom, mon, maap, syp, yg, bahagia, ..."
2,2022-10-04 03:23:26,1.577137e+18,RT @GOAL_ID: 𝗥𝗲𝘀𝗽𝗲𝗰𝘁 @realmadriden @osasuna_en...,Twitter for Android,0.0,47.0,goal_id realmadriden osasuna_en coehahqlk,"[goal_id, realmadriden, osasuna_en, coehahqlk]"
3,2022-10-04 03:23:25,1.577137e+18,RT @sulaimaneffendi: Dari Surabaya untuk Malan...,Twitter for iPhone,0.0,244.0,sulaimaneffendi dari surabaya untuk malang apa...,"[sulaimaneffendi, dari, surabaya, untuk, malan..."
4,2022-10-04 03:23:25,1.577137e+18,RT @GOAL_ID: 𝗥𝗲𝘀𝗽𝗲𝗰𝘁 @realmadriden @osasuna_en...,Twitter for Android,0.0,47.0,goal_id realmadriden osasuna_en coehahqlk,"[goal_id, realmadriden, osasuna_en, coehahqlk]"
5,2022-10-04 03:23:24,1.577137e+18,RT @CoachJustinL: Ga ada gunanya cari kesalaha...,Twitter for Android,0.0,170.0,coachjustinl ga ada gunanya cari kesalahan ini...,"[coachjustinl, ga, ada, gunanya, cari, kesalah..."
6,2022-10-04 03:23:23,1.577137e+18,Rest In Peace🥀🎗️Kejadian Kelam sejarah sepak b...,Twitter for Android,0.0,0.0,rest in peace kejadian kelam sejarah sepak bol...,"[rest, in, peace, kejadian, kelam, sejarah, se..."
7,2022-10-04 03:23:23,1.577137e+18,"RT @psisfcofficial: ""Kami Arema, Salam Satu Ji...",Twitter for Android,0.0,1290.0,psisfcofficial kami arema salam satu jiwa cupl...,"[psisfcofficial, kami, arema, salam, satu, jiw..."
8,2022-10-04 03:23:21,1.577137e+18,RT @Amar_0599: Sekolah hukum 4 tahun aja kadan...,Twitter for Android,0.0,826.0,amar_ sekolah hukum tahun aja kadang belum t...,"[amar_, sekolah, hukum, tahun, aja, kadang, be..."
9,2022-10-04 03:23:19,1.577137e+18,RT @IbnoeAhmad4: Tragedi Kanjuruhan yg dilakuk...,Twitter for Android,0.0,9319.0,ibnoeahmad tragedi kanjuruhan yg dilakukan ole...,"[ibnoeahmad, tragedi, kanjuruhan, yg, dilakuka..."


In [ ]:
normalized_word = pd.read_csv('https://raw.githubusercontent.com/masaboe/ams-2022/main/textProcessing/kamus_clean.csv')

In [ ]:
normalized_word.head

<bound method NDFrame.head of        Unnamed: 0    TIDAK BAKU                               BAKU
0               0       mangtab                              bagus
1               1   evolusionis                       revolusioner
2               6      abstarck                            abstrak
3               7  sebenenarnya                         sebenarnya
4               8   kataakanlah                         katakanlah
...           ...           ...                                ...
15017       16016     mendikbud  menteri pendidikan dan kebudayaan
15018       16017        mendag                menteri perdagangan
15019       16018       menaker               menteri tenaga kerja
15020       16019      memetwit                            mentwit
15021       16020      megangin                           memegang

[15022 rows x 3 columns]>

In [ ]:
normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

In [ ]:
df_new['final_tokens'] = df_new['tokens'].apply(normalized_term)

In [ ]:
i=0
final_string_tokens = []
for text in df_new['final_tokens'].values:
    EachReviewText = ""
    EachReviewText = ' '.join(text)
    final_string_tokens.append(EachReviewText)

In [ ]:
df_new["step02"] = final_string_tokens

In [ ]:
df_new.head(10)

,Datetime,Tweet Id,Text,Source,Likes,RT,cleanText,tokens,final_tokens,step02
0,2022-10-04 03:23:28,1.577137e+18,RT @dialoglukaku: Buat temen-temen boleh bantu...,Twitter for Android,0.0,2302.0,dialoglukaku buat temen temen boleh bantu retw...,"[dialoglukaku, buat, temen, temen, boleh, bant...","[dialoglukaku, buat, temen, temen, boleh, bant...",dialoglukaku buat temen temen boleh bantu retw...
1,2022-10-04 03:23:26,1.577137e+18,"RT @NarasiNewsroom: Mon maap, syp yg bahagia y...",Twitter for Android,0.0,5696.0,narasinewsroom mon maap syp yg bahagia pack kl...,"[narasinewsroom, mon, maap, syp, yg, bahagia, ...","[narasinewsroom, mon, maap, syp, yg, bahagia, ...",narasinewsroom mon maap syp yg bahagia pack kl...
2,2022-10-04 03:23:26,1.577137e+18,RT @GOAL_ID: 𝗥𝗲𝘀𝗽𝗲𝗰𝘁 @realmadriden @osasuna_en...,Twitter for Android,0.0,47.0,goal_id realmadriden osasuna_en coehahqlk,"[goal_id, realmadriden, osasuna_en, coehahqlk]","[goal_id, realmadriden, osasuna_en, coehahqlk]",goal_id realmadriden osasuna_en coehahqlk
3,2022-10-04 03:23:25,1.577137e+18,RT @sulaimaneffendi: Dari Surabaya untuk Malan...,Twitter for iPhone,0.0,244.0,sulaimaneffendi dari surabaya untuk malang apa...,"[sulaimaneffendi, dari, surabaya, untuk, malan...","[sulaimaneffendi, dari, surabaya, untuk, malan...",sulaimaneffendi dari surabaya untuk malang apa...
4,2022-10-04 03:23:25,1.577137e+18,RT @GOAL_ID: 𝗥𝗲𝘀𝗽𝗲𝗰𝘁 @realmadriden @osasuna_en...,Twitter for Android,0.0,47.0,goal_id realmadriden osasuna_en coehahqlk,"[goal_id, realmadriden, osasuna_en, coehahqlk]","[goal_id, realmadriden, osasuna_en, coehahqlk]",goal_id realmadriden osasuna_en coehahqlk
5,2022-10-04 03:23:24,1.577137e+18,RT @CoachJustinL: Ga ada gunanya cari kesalaha...,Twitter for Android,0.0,170.0,coachjustinl ga ada gunanya cari kesalahan ini...,"[coachjustinl, ga, ada, gunanya, cari, kesalah...","[coachjustinl, ga, ada, gunanya, cari, kesalah...",coachjustinl ga ada gunanya cari kesalahan ini...
6,2022-10-04 03:23:23,1.577137e+18,Rest In Peace🥀🎗️Kejadian Kelam sejarah sepak b...,Twitter for Android,0.0,0.0,rest in peace kejadian kelam sejarah sepak bol...,"[rest, in, peace, kejadian, kelam, sejarah, se...","[rest, in, peace, kejadian, kelam, sejarah, se...",rest in peace kejadian kelam sejarah sepak bol...
7,2022-10-04 03:23:23,1.577137e+18,"RT @psisfcofficial: ""Kami Arema, Salam Satu Ji...",Twitter for Android,0.0,1290.0,psisfcofficial kami arema salam satu jiwa cupl...,"[psisfcofficial, kami, arema, salam, satu, jiw...","[psisfcofficial, kami, arema, salam, satu, jiw...",psisfcofficial kami arema salam satu jiwa cupl...
8,2022-10-04 03:23:21,1.577137e+18,RT @Amar_0599: Sekolah hukum 4 tahun aja kadan...,Twitter for Android,0.0,826.0,amar_ sekolah hukum tahun aja kadang belum t...,"[amar_, sekolah, hukum, tahun, aja, kadang, be...","[amar_, sekolah, hukum, tahun, aja, kadang, be...",amar_ sekolah hukum tahun aja kadang belum ten...
9,2022-10-04 03:23:19,1.577137e+18,RT @IbnoeAhmad4: Tragedi Kanjuruhan yg dilakuk...,Twitter for Android,0.0,9319.0,ibnoeahmad tragedi kanjuruhan yg dilakukan ole...,"[ibnoeahmad, tragedi, kanjuruhan, yg, dilakuka...","[ibnoeahmad, tragedi, kanjuruhan, yg, dilakuka...",ibnoeahmad tragedi kanjuruhan yg dilakukan ole...


In [ ]:
df_new.to_csv('clean_dataset_kanjuruhan.csv',sep=";")